<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobart/kobart_summarization_to_torchscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 29.6 MB/s 
     |████████████████████████████████| 182 kB 69.9 MB/s 
     |████████████████████████████████| 7.6 MB 48.0 MB/s 


In [ ]:
import os
import torch
from torch.onnx import export
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')

sentence_text = '요약을 위해 작성하는 글입니다. 사실 이 글의 요점은 아무 주제도 없다는 것에 있습니다. 부디 기계가 이 요점을 잘 정리해서 보여주었으면 좋겠습니다.'
max_length = 512

model_input = tokenizer(
            sentence_text,
            return_tensors='np',
            truncation=True,
            max_length=max_length,
            padding="max_length",
            add_special_tokens=True
        )
input_ids = model_input["input_ids"]
attention_mask = model_input["attention_mask"]

tokens_tensor = torch.tensor(input_ids)
segments_tensors = torch.tensor(attention_mask)

class Pytorch_to_TorchScript(torch.nn.Module):
  def __init__(self):
    super(Pytorch_to_TorchScript, self).__init__()
    self.model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization').cuda()
  def forward(self, data, attention_mask=None):
    return self.model.generate(data.cuda(), num_beams=4,  max_length=512,  eos_token_id=1)
  



You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:

# Creating the trace
pt_model = Pytorch_to_TorchScript().eval()
traced_model = torch.jit.trace(pt_model, (tokens_tensor, segments_tensors))
torch.jit.save(traced_model, "traced_bart.pt")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.8/dist-packages/transformers/models/bart/modeling_bart.py:232: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.8/dist-packages/transformers/models/bart/modeling_bart.py:239: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.size() != (bsz, 1, tgt_len, src_len):
/usr/local/lib/pyt

In [ ]:
!ls

sample_data  traced_bart.pt


In [ ]:
loaded_model = torch.jit.load("traced_bart.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=Pytorch_to_TorchScript
  (model): RecursiveScriptModule(
    original_name=BartForConditionalGeneration
    (model): RecursiveScriptModule(
      original_name=BartModel
      (shared): RecursiveScriptModule(original_name=Embedding)
      (encoder): RecursiveScriptModule(
        original_name=BartEncoder
        (embed_tokens): RecursiveScriptModule(original_name=Embedding)
        (embed_positions): RecursiveScriptModule(original_name=BartLearnedPositionalEmbedding)
        (layers): RecursiveScriptModule(
          original_name=ModuleList
          (0): RecursiveScriptModule(
            original_name=BartEncoderLayer
            (self_attn): RecursiveScriptModule(
              original_name=BartAttention
              (k_proj): RecursiveScriptModule(original_name=Linear)
              (v_proj): RecursiveScriptModule(original_name=Linear)
              (q_proj): RecursiveScriptModule(original_name=Linear)
              (out_proj): RecursiveSc

In [ ]:
max_length = 512
sentence_text = """9일 국토교통부 실거래가 공개시스템 등에 따르면, 광교중흥S클래스 전용 109㎡는 지난달 17일 17억5000만원에 매매됐다. 이보다 한 달 전인 10월 18일에는 26억2000만원에 직거래된 적이 있다.

이 아파트 전용 109㎡는 지난해 6월 22일 27억원에 거래돼 최고점을 찍었다. 이후 10개월가량 매매가 없다가, 올해 4월과 5월 각각 20억2000만~20억3000만원에 손바뀜됐다. 그러다 다섯 달만에 돌연 6억 오른 가격에 직거래됐다가, 다시 가격이 크게 떨어진 것이다. 전용 85㎡도 지난해 7~11월에는 18억원에 매매가 이뤄졌으나, 이후 꾸준히 하락세를 보였다. 지난달에는 고점 대비 6억3000만원 빠진 11억7000만원에 거래됐다.

10억원 가까이 빠진 가격에 매매가 이뤄졌다는 소식이 알려지자 부동산 커뮤니티들이 달아올랐다. 한 네티즌은 “서울도 아니고 40평대가 20억원을 넘는 것은 말이 안 되는 거품이었다”고 했다. 다른 네티즌은 “(10억원이 내렸지만) 아직도 비싸다”고 했다. “분양가를 감안하면 (기존 최고가보다) 10억원가량 싸게 팔아도 남는 장사”라는 이들도 있었다. 반면 입주민으로 보이는 네티즌들은 “전국적으로 하락장이고, 1주택 실거주라 (집값이) 내려도 부담이 없다. 마음 편하게 사는 게 최고”라고 했다. “급등했으니 급락하는 것도 당연하다”며 동조하는 이들도 있었다.

한국부동산원에 따르면, 광교신도시가 포함된 수원 영통구의 올해 집값 누적하락률은 12.53%에 달한다. 광교 뿐만 아니라 경기 남부의 다른 신도시인 동탄 역시 집값이 하락세를 보이고 있다. 동탄 더샵레이크 에듀타운 전용 84㎡는 지난달 15일 7억원에 거래됐다. 최고가인 12억1700만원에 비교하면 5억원 이상 낮다. 힐스테이트동탄 84㎡는 지난달 18일 5억8000만원에 매매가 이뤄졌다. 이전 최고가는 9억6500만원이었다. 동탄이 포함된 화성의 올해 집값 누적 하락률은 10.92%였다.

아파트 시장의 침체기는 이어지고 있다. 8일 한국부동산원이 발표한 주간 아파트 가격동향(5일 기준)을 보면, 이번주 전국 아파트 매매가격은 전주 대비 0.59% 떨어졌다. 2012년 한국부동산원 시세 조사 이래 가장 큰폭의 하락세다. 서울은 0.59%, 경기는 0.78% 내렸다. 주택가격 추가 하락 우려와 금리인상에 대한 부담으로 매수 문의가 거의 없는 상황이 이어지는 가운데, 일부 급매물이 소화되면서 가격이 전반적으로 떨어지고 있다는 진단이다."""
model_input = tokenizer(
            sentence_text,
            return_tensors='np',
            truncation=True,
            max_length=max_length,
            padding="max_length",
            add_special_tokens=True
        )
input_ids = model_input["input_ids"]
attention_mask = model_input["attention_mask"]

tokens_tensor = torch.tensor(input_ids)
segments_tensors = torch.tensor(attention_mask)

In [ ]:
loaded_model = torch.jit.load("traced_bart.pt")
loaded_model.eval()

dummy_input = [tokens_tensor, segments_tensors]

result = loaded_model(*dummy_input)

In [ ]:
result.shape

torch.Size([1, 93])

In [ ]:
tokenizer.decode(result.squeeze().tolist(), skip_special_tokens=True)

'국토교통부 실거래가 공개시스템 등에 따르면, 광교중흥S클래스 전용 109m2는 지난달 17일 17억5000만원에 매매됐다. 이보다 한 달 전인 10월 18일에는 26억2000만원에 직거래된 적이 있다. 이후 10개월가량 매매가 없다가, 올해 4월과 5월 각각 20억2000만~20억3000만원에 손바뀜됐다가, 다시 가격이 크게 떨어진 것이다. 전용 85m2도'

# config.pbtxt

```text
name: "kobart_summarization"
platform: "pytorch_libtorch"
input [
 {
    name: "input__0"
    data_type: TYPE_INT32
    dims: [1, 512]
  } ,
{
    name: "input__1"
    data_type: TYPE_INT32
    dims: [1, 512]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [1, 3]
  }
```